# SQL 2 - multiple tables

In [1]:
import mysql.connector
import pandas as pd

In [2]:
import mysql.connector
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()  # loads variables from .env file
mysql_password = os.getenv("MYSQL_PASSWORD")

connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password=mysql_password,
)
cursor = connection.cursor()

In [3]:
cursor.execute("CREATE DATABASE IF NOT EXISTS Accounts;")
cursor.execute("USE Accounts;")

In [4]:
cursor.execute("DROP TABLE IF EXISTS Products, Customers, Orders;")
cursor.execute("CREATE TABLE Products (id INT AUTO_INCREMENT PRIMARY KEY, item VARCHAR(20), price INT, stockQty INT);")
cursor.execute("CREATE TABLE Customers (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(20), location VARCHAR(20));")
cursor.execute("CREATE TABLE Orders (id INT AUTO_INCREMENT PRIMARY KEY, customerId INT, itemId INT, qty INT, FOREIGN KEY (customerId) REFERENCES Customers(id), FOREIGN KEY (itemId) REFERENCES Products(id));")

In [5]:
cursor.execute('INSERT INTO Products (item, price, stockQty) VALUES '
               '("Teapot", 10, 25),'
               '("Bicycle", 450, 3),'
               '("Dustpan", 4, 30);')
cursor.execute('INSERT INTO Customers (name, location) VALUES '
               '("Pablo", "London"),'
               '("Sophie", "Liphook"),'
               '("Sabina", "London");')
cursor.execute('INSERT INTO Orders (customerId, itemId, qty) VALUES '
               '(1, 1, 1),'
               '(1, 2, 1),'
               '(2, NULL, 1),'
               '(1, 3, NULL),'
               '(2, 1, 1);')
connection.commit()

In [6]:
# Join Orders with Customers and Products
cursor.execute("""
               SELECT c.name AS Customer, p.item AS Item, o.qty AS Quantity
               FROM Orders o
                        JOIN Customers c ON o.customerId = c.id
                        JOIN Products p ON o.itemId = p.id
               ORDER BY c.name, p.item;
               """)

# Fetch results
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

# Convert to DataFrame for nice display
df = pd.DataFrame(rows, columns=columns)
df#

,Customer,Item,Quantity
0,Pablo,Bicycle,1.0
1,Pablo,Dustpan,NaN
2,Pablo,Teapot,1.0
3,Sophie,Teapot,1.0


## INNER JOIN

This removes any ```Customers``` with no orders and any ```Products``` that have not been ordered.

NOTE: There was no need to explicitly write ```INNER``` as it is the default.

In [7]:
cursor.execute("""
               SELECT c.name AS Customer, p.item AS Item, o.qty AS Quantity
               FROM Orders o
                        INNER JOIN Customers c ON o.customerId = c.id
                        INNER JOIN Products p ON o.itemId = p.id;
               """)

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df


,Customer,Item,Quantity
0,Pablo,Teapot,1.0
1,Pablo,Bicycle,1.0
2,Pablo,Dustpan,NaN
3,Sophie,Teapot,1.0


## LEFT JOIN

The ```LEFT``` table is the one that comes before the ```JOIN``` keyword, e.g. ```FROM Customers LEFT JOIN``` ...

In [8]:
# ALL Customers are kept, even if they have no orders - there is one row for each order and an extra for Sabina who has no orders
cursor.execute("""
               SELECT c.name AS Customer, p.item AS Item, o.qty AS Quantity
               FROM Customers c
                        LEFT JOIN Orders o ON c.id = o.customerId
                        LEFT JOIN Products p ON o.itemId = p.id;
               """)

df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df


,Customer,Item,Quantity
0,Pablo,Teapot,1.0
1,Pablo,Bicycle,1.0
2,Pablo,Dustpan,NaN
3,Sophie,None,1.0
4,Sophie,Teapot,1.0
5,Sabina,None,NaN


In [10]:
# ALL Customers are kept, even if they have no orders - there is one row for each order and an extra for Sabina who has no orders
cursor.execute("""
               SELECT c.name AS Customer, p.item AS Item, o.qty AS Quantity
               FROM Customers c
                        LEFT JOIN Orders o ON c.id = o.customerId
                        INNER JOIN Products p ON o.itemId = p.id;
               """)

df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df


,Customer,Item,Quantity
0,Pablo,Teapot,1.0
1,Pablo,Bicycle,1.0
2,Pablo,Dustpan,NaN
3,Sophie,Teapot,1.0


## RIGHT JOIN

Keeps all ```Products``` even if they are not ordered - contains ```Order``` lines with no ```Product```

In [11]:
cursor.execute("""
               SELECT p.item AS Item, c.name AS Customer, o.qty AS Quantity
               FROM Products p
                        RIGHT JOIN Orders o ON p.id = o.itemId
                        RIGHT JOIN Customers c ON o.customerId = c.id;
               """)

df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df


,Item,Customer,Quantity
0,Teapot,Pablo,1.0
1,Bicycle,Pablo,1.0
2,Dustpan,Pablo,NaN
3,None,Sophie,1.0
4,Teapot,Sophie,1.0
5,None,Sabina,NaN
